## Get Tollywood Movies

In [39]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import sys

In [2]:
## Developing the base code to ensure whethe all the details are getting parsed or not

def scrape_movie_details_Telugu(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Telugu"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_telugu = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series&release_date=1990-01-01,2024-12-31&user_rating=6.5,&num_votes=7000,&primary_language=te&sort=alpha,asc"


    # Set up Selenium WebDriver
driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

try:
    # Locate the "21 more" button using XPath
    see_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '21 more')]"))
    )

    # Scroll the page to ensure the button is visible
    driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
    time.sleep(1)  # Optional sleep to ensure the page scrolls completely

    # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
    driver.execute_script("arguments[0].click();", see_more_button)
#     print("Clicked 'See More' button.")
    time.sleep(5)  # Wait for additional content to load

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items[0:3]:  # limited to 3 movies to test the code
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Telugu(movie_url)
        movies_list_telugu.append(details)
        print(details)
        
# telugu_movies_df= pd.DataFrame(movies_list_telugu)
# telugu_movies_df
        
# close the driver

driver.quit()

{'Movie Name': 'Original title: 1 - Nenokkadine', 'Movie Release Year': '2014', 'Runtime': '170 mins', 'Rating': '4.0', 'Director': 'Sukumar', 'Description': "A rock star must overcome his psychological inhibitions to seek revenge for his parents' death.", 'Cast': ['Mahesh Babu', 'Kriti Sanon', 'Nassar'], 'Genre': ['Action', 'Thriller'], 'Country': 'India', 'Language': 'Telugu'}
{'Movie Name': 'Agent Sai Srinivasa Athreya', 'Movie Release Year': '2019', 'Runtime': '148 mins', 'Rating': '4.15', 'Director': 'Swaroop Rsj', 'Description': 'Agent Sai Srinivasa Athreya is an authentic humorous investigative thriller revolving around the adventures of a detective based out of Nellore.', 'Cast': ['Naveen Polishetty', 'Shruti Sharma', 'Krishneswara Rao'], 'Genre': ['Comedy', 'Crime', 'Mystery', 'Thriller'], 'Country': 'India', 'Language': 'Telugu'}
{'Movie Name': 'Akhanda', 'Movie Release Year': '2021', 'Runtime': '167 mins', 'Rating': '3.4', 'Director': 'Boyapati Srinu', 'Description': 'When t

In [36]:
def scrape_movie_details_Telugu(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Telugu"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_telugu = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series&release_date=1990-01-01,2024-12-31&user_rating=6.5,&num_votes=7000,&primary_language=te&sort=alpha,asc"


    # Set up Selenium WebDriver
driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import sys

try:
    # Locate the "21 more" button using XPath
    see_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '21 more')]"))
    )

    # Scroll the page to ensure the button is visible
    driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
    time.sleep(1)  # Optional sleep to ensure the page scrolls completely

    # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
    driver.execute_script("arguments[0].click();", see_more_button)
#     print("Clicked 'See More' button.")
    time.sleep(5)  # Wait for additional content to load

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items:  # updateing to get all the movies details
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Telugu(movie_url)
        movies_list_telugu.append(details)
        sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list_telugu)}")
        sys.stdout.flush()

        
#         print(len(movies_list_telugu))
#         print(details)
        
telugu_movies_df = pd.DataFrame(movies_list_telugu)
telugu_movies_df.index = range(1, len(telugu_movies_df) + 1)

        
# close the driver

driver.quit()

Number of movies scraped: 71

In [37]:
telugu_movies_df = telugu_movies_df.sort_values(by="Movie Release Year", ascending=True)
telugu_movies_df.index = range(1,len(telugu_movies_df)+1)

In [38]:
telugu_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,Okkadu,2003,171 mins,4.0,Gunasekhar,A Kabbadi player rescues a young woman from an...,"[Mahesh Babu, Bhoomika Chawla, Prakash Raj]","[Action, Drama, Romance, Sport]",India,Telugu
2,Arya,2004,151 mins,3.9,Sukumar,"Geethanjali, who is already the girlfriend of ...","[Allu Arjun, Anuradha Mehta, Siva Balaji]","[Action, Comedy, Drama, Romance]",India,Telugu
3,Athadu,2005,172 mins,4.1,Trivikram Srinivas,"A gunman for hire is framed for murder, and as...","[Mahesh Babu, Trisha Krishnan, Sonu Sood]","[Action, Crime, Drama, Mystery, Thriller]",India,Telugu
4,Bommarillu,2006,170 mins,4.1,Bommarillu Baskar,A young man realizes that he has to push back ...,"[Siddharth, Genelia Deshmukh, Prakash Raj]","[Comedy, Drama, Romance]",India,Telugu
5,Pokiri,2006,164 mins,4.0,Puri Jagannadh,A money-minded criminal has to deal with two r...,"[Mahesh Babu, Ileana D'Cruz, Prakash Raj]","[Gun Fu, Action, Crime, Thriller]",India,Telugu
...,...,...,...,...,...,...,...,...,...,...
67,Chakravyuham: The Trap,2023,111 mins,3.25,Chetkuri Madhusudhan,"Siri, a business partner and a housewife, was ...","[Vivek Trivedi, Ajay, Bhargav]","[Crime, Mystery, Thriller]",India,Telugu
68,Hanu Man,2024,158 mins,3.9,Prasanth Varma,An imaginary place called Anjanadri where the ...,"[Teja Sajja, Amritha Aiyer, Varalaxmi Sarathku...","[Superhero, Action, Adventure, Fantasy]",India,Telugu
69,Original title: Kalki 2898-AD,2024,180 mins,3.5,Nag Ashwin,The future of those in the dystopian city of K...,"[Prabhas, Amitabh Bachchan, Kamal Haasan]","[Action Epic, Desert Adventure, Dystopian Sci-...",India,Telugu
70,Saripodhaa Sanivaaram,2024,166 mins,3.5,Vivek Athreya,In order to defend the innocent from a corrupt...,"[Nani, S.J. Suryah, Priyanka Arulmohan]","[Action, Thriller]",India,Telugu


## Get Bollywood movies

In [6]:
def scrape_movie_details_Bollywood(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Hindi"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_Bollywood = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,2024-12-31&user_rating=7,10&num_votes=10000,&primary_language=hi,mr,bho,or,pa&moviemeter=,20000&sort=alpha,asc"
    # Set up Selenium WebDriver
driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import sys

# Locate and extract the "See More" button for 50 more or 10 more using BeautifulSoup
try:
    # Locate the "21 more" button using XPath
    see_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '41 more')]"))
    )

    # Scroll the page to ensure the button is visible
    driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
    time.sleep(1)  # Optional sleep to ensure the page scrolls completely

    # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
    driver.execute_script("arguments[0].click();", see_more_button)
#     print("Clicked 'See More' button.")
    time.sleep(5)  # Wait for additional content to load

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items:  # updateing to get all the movies details
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Bollywood(movie_url)
        movies_list_Bollywood.append(details)
        sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list_Bollywood)}")
        sys.stdout.flush()

        
#         print(len(movies_list_Bollywood))
#         print(details)
        
        
# close the driver

driver.quit()

Number of movies scraped: 91

In [7]:
Bollywood_movies_df = pd.DataFrame(movies_list_Bollywood)
Bollywood_movies_df.index = range(1, len(Bollywood_movies_df) + 1)


In [8]:
Bollywood_movies_df = Bollywood_movies_df.sort_values(by="Movie Release Year", ascending=True)
Bollywood_movies_df.index = range(1,len(Bollywood_movies_df)+1)

In [9]:
Bollywood_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,Andaz Apna Apna,1994,160 mins,4.0,Rajkumar Santoshi,Two slackers competing for the affections of a...,"[Aamir Khan, Salman Khan, Raveena Tandon]","[Parody, Slapstick, Action, Comedy, Romance]",India,Hindi
2,Dilwale Dulhania Le Jayenge,1995,181 mins,4.0,Aditya Chopra,"When Raj meets Simran in Europe, it isn't love...","[Shah Rukh Khan, Kajol, Amrish Puri]","[Drama, Romance]",India,Hindi
3,Dil To Pagal Hai,1997,No runtime found,3.5,Yash Chopra,A grand musical about three passionate and dre...,"[Shah Rukh Khan, Madhuri Dixit, Karisma Kapoor]","[Comedy, Drama, Musical, Romance]",India,Hindi
4,Kuch Kuch Hota Hai,1998,185 mins,3.75,Karan Johar,"During their college years, Anjali was in love...","[Shah Rukh Khan, Kajol, Rani Mukerji]","[Comedy, Drama, Musical, Romance]",India,Hindi
5,Dil Se..,1998,163 mins,3.75,Mani Ratnam,A radio broadcaster's intense attraction to a ...,"[Shah Rukh Khan, Manisha Koirala, Preity G Zinta]","[Drama, Romance]",India,Hindi
...,...,...,...,...,...,...,...,...,...,...
87,Maidaan,2024,181 mins,4.0,Amit Ravindernath Sharma,Inspired by the Indian national football team ...,"[Ajay Devgn, Sharik Khan Jr., Priyamani]","[Docudrama, Period Drama, Soccer, Biography, D...",India,Hindi
88,Article 370,2024,158 mins,3.9,Aditya Suhas Jambhale,"Ahead of a major constitutional decision, spec...","[Yami Gautam, Priyamani, Vaibhav Tatwawadi]","[Political Drama, Political Thriller, Action, ...",India,Hindi
89,Sector 36,2024,123 mins,3.55,Aditya Nimbalkar,"A fictional story Inspired by true events, sev...","[Vikrant Massey, Deepak Dobriyal, Akash Khurana]",[Thriller],India,Hindi
90,Stree 2: Sarkate Ka Aatank,2024,147 mins,3.5,Amar Kaushik,"After the events of Stree, the town of Chander...","[Bhumi Rajgor, Shruti Pandey, Sunil Kumar]","[Comedy, Horror]",India,Hindi


In [10]:
Bollywood_movies_df["Movie Name"].values==['No title found']

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

## Get Kannada Movies

In [11]:
def scrape_movie_details_Kannada(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Kannada"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_Kannada = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_series&release_date=1990-01-01,2024-12-31&user_rating=6.5,&num_votes=7000,&primary_language=kn&sort=alpha,asc"
    # Set up Selenium WebDriver
driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import sys

# Locate and extract the "See More" button for 50 more or 10 more using BeautifulSoup
try:
    if True:
        # Locate the "21 more" button using XPath
        see_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '41 more')]"))
        )

        # Scroll the page to ensure the button is visible
        driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
        time.sleep(1)  # Optional sleep to ensure the page scrolls completely

        # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
        driver.execute_script("arguments[0].click();", see_more_button)
    #     print("Clicked 'See More' button.")
        time.sleep(5)  # Wait for additional content to load
    else:
        pass

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")
    pass  # Ensure the program continues to the next stage

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items:  # updateing to get all the movies details
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Kannada(movie_url)
        movies_list_Kannada.append(details)
        sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list_Kannada)}")
        sys.stdout.flush()

        
#         print(len(movies_list_Kannada))
#         print(details)
        
        
# close the driver

driver.quit()

Error clicking 'See More' button: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AC1E38A5+3004357]
	(No symbol) [0x00007FF7ABE79970]
	(No symbol) [0x00007FF7ABD2582A]
	(No symbol) [0x00007FF7ABD75B8E]
	(No symbol) [0x00007FF7ABD75E7C]
	(No symbol) [0x00007FF7ABDBEC27]
	(No symbol) [0x00007FF7ABD9BC1F]
	(No symbol) [0x00007FF7ABDBBA4C]
	(No symbol) [0x00007FF7ABD9B983]
	(No symbol) [0x00007FF7ABD67628]
	(No symbol) [0x00007FF7ABD68791]
	GetHandleVerifier [0x00007FF7AC20A00D+3161901]
	GetHandleVerifier [0x00007FF7AC25E060+3506048]
	GetHandleVerifier [0x00007FF7AC25400D+3465005]
	GetHandleVerifier [0x00007FF7ABFD0EEB+830987]
	(No symbol) [0x00007FF7ABE8467F]
	(No symbol) [0x00007FF7ABE809D4]
	(No symbol) [0x00007FF7ABE80B6D]
	(No symbol) [0x00007FF7ABE70149]
	BaseThreadInitThunk [0x00007FFC329A257D+29]
	RtlUserThreadStart [0x00007FFC3386AF08+40]

Number of movies scraped: 19

In [12]:
Kannada_movies = pd.DataFrame(movies_list_Kannada)
Kannada_movies.head()

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
0,777 Charlie,2022,164 mins,4.35,Kiranraj K,Dharma is stuck in a rut with his negative and...,"[Rakshit Shetty, Charlie, Sangeetha Sringeri]","[Adventure, Comedy, Drama]",India,Kannada
1,Avane Srimannarayana,2019,186 mins,3.85,Sachin Ravi,In the pursuit of solving an ancient mystery o...,"[Rakshit Shetty, Shanvi Srivastava, Achyuth Ku...","[Action, Adventure, Comedy, Drama, Fantasy]",India,Kannada
2,Dia,2020,137 mins,4.0,K.S. Ashoka,Dia takes three years to confess her feelings ...,"[Kushi, Kushee Ravi, Pruthvi Ambaar]","[Drama, Romance]",India,Kannada
3,Garuda Gamana Vrishabha Vahana,2021,151 mins,4.1,Raj B. Shetty,Set in the backdrop of the coastal and cultura...,"[Raj B. Shetty, Rishab Shetty, Gopalkrishna De...","[Crime, Drama, Thriller]",India,Kannada
4,James,2022,149 mins,3.7,Chethan Kumar,"James also called Santhosh Kumar, who works as...","[Puneeth Rajkumar, Priya Anand, Meka Srikanth]","[Action, Comedy, Drama]",India,Kannada


In [13]:
Kannada_movies_df = Kannada_movies.sort_values(by = "Movie Release Year", ascending=True)
Kannada_movies_df.index = range(1,len(Kannada_movies_df)+1)
Kannada_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,Lucia,2013,135 mins,4.15,Pawan Kumar,A man suffering from insomnia is tricked into ...,"[Sathish Neenasam, Sruthi Hariharan, Achyuth K...","[Drama, Sci-Fi, Thriller]",India,Kannada
2,Ulidavaru Kandanthe,2014,154 mins,4.2,Rakshit Shetty,A journalist sets out to uncover the truth beh...,"[Rakshit Shetty, Rishab Shetty, Kishore Kumar G.]","[Crime, Drama, Thriller]",India,Kannada
3,Ugramm,2014,132 mins,4.0,Prashanth Neel,A man with a dangerous past must protect a gir...,"[Sri Murali, Haripriya, Thilak Shekar]","[Action, Crime, Drama, Thriller]",India,Kannada
4,RangiTaranga,2015,149 mins,4.05,Anup Bhandari,Mysterious events begin to unfold after a recl...,"[Nirup Bhandari, Radhika Narayan, Avantika She...","[Mystery, Thriller]",India,Kannada
5,Uppi 2,2015,135 mins,3.9,Upendra,A psychological thriller which depicts the fac...,"[Upendra, Kristina Akheeva, Parul Yadav]","[Action, Mystery, Thriller]",India,Kannada
6,Kirik Party,2016,159 mins,4.1,Rishab Shetty,Kirik Party is the story of a gang of mischiev...,"[Rakshit Shetty, Rashmika Mandanna, Samyuktha ...","[Comedy, Drama, Romance]",India,Kannada
7,K.G.F: Chapter 1,2018,154 mins,4.1,Prashanth Neel,"In the 1970s, a gangster named Rocky goes unde...","[Yash, Srinidhi Shetty, Ramachandra Raju]","[Action Epic, Epic, Gangster, Period Drama, Ac...",India,Kannada
8,Avane Srimannarayana,2019,186 mins,3.85,Sachin Ravi,In the pursuit of solving an ancient mystery o...,"[Rakshit Shetty, Shanvi Srivastava, Achyuth Ku...","[Action, Adventure, Comedy, Drama, Fantasy]",India,Kannada
9,Dia,2020,137 mins,4.0,K.S. Ashoka,Dia takes three years to confess her feelings ...,"[Kushi, Kushee Ravi, Pruthvi Ambaar]","[Drama, Romance]",India,Kannada
10,Garuda Gamana Vrishabha Vahana,2021,151 mins,4.1,Raj B. Shetty,Set in the backdrop of the coastal and cultura...,"[Raj B. Shetty, Rishab Shetty, Gopalkrishna De...","[Crime, Drama, Thriller]",India,Kannada


## Get Tamil Movies

In [14]:
def scrape_movie_details_Tamil(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Tamil"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_Tamil = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,2024-12-31&user_rating=7,10&num_votes=8000,&primary_language=ta&sort=alpha,asc"    # Set up Selenium WebDriver
driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import sys

# Locate and extract the "See More" button for 50 more or 10 more using BeautifulSoup
try:
    if True:
        # Locate the "21 more" button using XPath
        see_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '41 more') or contains(text(), '38 more') or contains(text(), '21 more')]"))
    )

        # Scroll the page to ensure the button is visible
        driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
        time.sleep(1)  # Optional sleep to ensure the page scrolls completely

        # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
        driver.execute_script("arguments[0].click();", see_more_button)
    #     print("Clicked 'See More' button.")
        time.sleep(5)  # Wait for additional content to load
    else:
        pass

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")
    pass  # Ensure the program continues to the next stage

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items:  # updateing to get all the movies details
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Tamil(movie_url)
        movies_list_Tamil.append(details)
        sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list_Tamil)}")
        sys.stdout.flush()

        
#         print(len(movies_list_Tamil))
#         print(details)
        
        
# close the driver

driver.quit()

Number of movies scraped: 88

In [15]:
Tamil_movies_df = pd.DataFrame(movies_list_Tamil)
Tamil_movies_df = Tamil_movies_df.sort_values(by = "Movie Release Year", ascending=True)
Tamil_movies_df.index = range(1,len(Tamil_movies_df)+1)
Tamil_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,Thalapathi,1991,157 mins,4.25,Mani Ratnam,An orphan named Surya raised in a slum befrien...,"[Rajinikanth, Mammootty, Shobana]","[Action, Crime, Drama]",India,Tamil
2,Thevar Magan,1992,145 mins,4.35,Bharathan,The urbane son of a village chieftain struggle...,"[Kamal Haasan, Shivaji Ganesan, Nassar]","[Action, Drama, Romance]",India,Tamil
3,Roja,1992,137 mins,4.05,Mani Ratnam,A woman from a village in Tamil Nadu marries a...,"[Arvind Swamy, Madhoo, Pankaj Kapur]","[Drama, Romance, Thriller]",India,Tamil
4,Bombay,1995,130 mins,4.05,Mani Ratnam,A Hindu man and a Muslim woman fall in love in...,"[Arvind Swamy, Manisha Koirala, Nassar]","[Drama, Romance]",India,Tamil
5,Baasha,1995,145 mins,4.15,Suresh Krishna,"An auto driver, Manikam, desperately tries to ...","[Rajinikanth, Nagma, Raghuvaran]","[Action, Crime, Drama]",India,Tamil
...,...,...,...,...,...,...,...,...,...,...
84,Maaveeran,2023,163 mins,3.7,Madonne Ashwin,A cowardly cartoonist starts being 'controlled...,"[Sivakarthikeyan, Aditi Shankar, Mysskin]","[Action, Comedy, Drama, Fantasy, Thriller]",India,Tamil
85,Maharaja,2024,141 mins,4.25,Nithilan Saminathan,A barber seeks vengeance after his home is bur...,"[Vijay Sethupathi, Anurag Kashyap, Mamta Mohan...","[Action, Crime, Drama, Thriller]",India,Tamil
86,Vettaiyan,2024,163 mins,3.75,T.J. Gnanavel,Ruthless criminal activity is met with force f...,"[Rajinikanth, Amitabh Bachchan, Fahadh Faasil]","[Cop Drama, One-Person Army Action, Action, Cr...",India,Tamil
87,Amaran,2024,167 mins,4.25,Rajkumar Periasamy,The life of Major Mukund Varadarajan and is se...,"[Sivakarthikeyan, Sai Pallavi, Bhuvan Arora]","[Action Epic, Action, Biography, Drama, War]",India,Tamil


## Get Malayalam Movies

In [16]:
def scrape_movie_details_Malayalam(movie_url):
    driver.get(movie_url)
    time.sleep(5)
    
    movie_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Getting movie name
    if movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx'):
        movie_name = movie_soup.find('div', class_='sc-ec65ba05-1 fUCCIx').text.strip()
    elif movie_soup.find('span', class_='hero__primary-text'):
        movie_name = movie_soup.find('span', class_='hero__primary-text').text.strip()
    else:
        movie_name = "No title found"
    
    # Getting movie release year
    release_year_tag = movie_soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt')
    if release_year_tag:
        release_year = release_year_tag.find('li', class_='ipc-inline-list__item').find('a', class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color').text.strip()
    else:
        release_year = "No release year found"
    
    # Getting runtime
    runtime_tag = movie_soup.find("li", class_="ipc-inline-list__item", string= (re.compile(r'\d+h(\s\d+m)?')))
    if runtime_tag:
        runtime_text = runtime_tag.text.strip()
        match = re.match(r'(\d+)h(?: (\d+)m)?', runtime_text)
        if match:
            # Extract hours and minutes, with minutes defaulting to 0 if not present
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            runtime = str((hours * 60) + minutes) + ' mins'
        else:
            runtime = "Runtime format not recognized."

    else:
        runtime = "No runtime found"
    
    # Getting Rating out of 5 where the rating was given was out of 10
    if movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf'):
        rating = movie_soup.find('span', class_ = 'sc-d541859f-1 imUuxf').text.strip()  
        rating = str(float(rating)/2)
    else:
        rating = "No rating found"
    # Getting Director
    director = movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text.strip() if movie_soup.find('a', class_ = 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link') else "No Director found"
    
    # Getting Movie Description
    description = movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB').text.strip() if movie_soup.find('span', class_ = 'sc-3ac15c8d-0 hRUoSB') else "No Description found"
    
    # Getting movie imp cast
    cast_soup = movie_soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    cast_container = cast_soup[1].find('div', class_='ipc-metadata-list-item__content-container')
    cast_elements = cast_container.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    cast_list = [cast_element.text.strip() for cast_element in cast_elements]
    main_cast = cast_list if cast_list else "No Cast found"
    
    # Getting Genre of the movie
    genre_soup = movie_soup.find('div', class_='ipc-chip-list__scroller')
    genre_tags = genre_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre_list = [genre_tag.text.strip() for genre_tag in genre_tags]
    genre = genre_list if genre_list else "No Genre found"
    
    # Getting Country & Language
    country = "India"
    language = "Malayalam"
    
    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

    
# Creating the empty list
movies_list_Malayalam = []

# URL to scrape
url = f"https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,2024-12-31&user_rating=6.5,10&num_votes=7000,&primary_language=ml&sort=alpha,asc"

driver = webdriver.Chrome()  

# Open the main webpage
driver.get(url)

# Wait for the page to load fully
time.sleep(5)

# Try to click the "21 more" button if present
# importing the req lib
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
import sys

# Locate and extract the "See More" button for 50 more or 10 more using BeautifulSoup
try:
    if True:
        # Locate the "21 more" button using XPath
        see_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), '41 more') or contains(text(), '38 more') or contains(text(), '21 more')]"))
    )

        # Scroll the page to ensure the button is visible
        driver.execute_script("arguments[0].scrollIntoView();", see_more_button)
        time.sleep(1)  # Optional sleep to ensure the page scrolls completely

        # Use JavaScript to click the "21 more" button (in case Selenium click doesn't work)
        driver.execute_script("arguments[0].click();", see_more_button)
    #     print("Clicked 'See More' button.")
        time.sleep(5)  # Wait for additional content to load
    else:
        pass

except Exception as e:
    print(f"Error clicking 'See More' button: {e}")
    pass  # Ensure the program continues to the next stage

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")
# print(soup)

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "ipc-metadata-list-summary-item"})

# Extract and print details for each movie
for item in list_items:  # updateing to get all the movies details
    # Extract movie link
    a_tag = item.find("a")
    movie_url = f"https://www.imdb.com{a_tag['href']}"
#     print(movie_url)
    if movie_url:
        # Scrape details from the movie page
        details = scrape_movie_details_Malayalam(movie_url)
        movies_list_Malayalam.append(details)
        sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list_Malayalam)}")
        sys.stdout.flush()

        
#         print(len(movies_list_Malayalam))
#         print(details)
        
        
# close the driver

driver.quit()

Error clicking 'See More' button: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AC1E38A5+3004357]
	(No symbol) [0x00007FF7ABE79970]
	(No symbol) [0x00007FF7ABD2582A]
	(No symbol) [0x00007FF7ABD75B8E]
	(No symbol) [0x00007FF7ABD75E7C]
	(No symbol) [0x00007FF7ABDBEC27]
	(No symbol) [0x00007FF7ABD9BC1F]
	(No symbol) [0x00007FF7ABDBBA4C]
	(No symbol) [0x00007FF7ABD9B983]
	(No symbol) [0x00007FF7ABD67628]
	(No symbol) [0x00007FF7ABD68791]
	GetHandleVerifier [0x00007FF7AC20A00D+3161901]
	GetHandleVerifier [0x00007FF7AC25E060+3506048]
	GetHandleVerifier [0x00007FF7AC25400D+3465005]
	GetHandleVerifier [0x00007FF7ABFD0EEB+830987]
	(No symbol) [0x00007FF7ABE8467F]
	(No symbol) [0x00007FF7ABE809D4]
	(No symbol) [0x00007FF7ABE80B6D]
	(No symbol) [0x00007FF7ABE70149]
	BaseThreadInitThunk [0x00007FFC329A257D+29]
	RtlUserThreadStart [0x00007FFC3386AF08+40]

Number of movies scraped: 46

In [17]:
Malayalam_movies_df = pd.DataFrame(movies_list_Malayalam)
Malayalam_movies_df = Malayalam_movies_df.sort_values(by = "Movie Release Year", ascending=True)
Malayalam_movies_df.index = range(1, len(Malayalam_movies_df)+1)
Malayalam_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,Original title: Manichitrathazhu,1993,169 mins,4.35,Fazil,When a forbidden room in an old bungalow is un...,"[Mohanlal, Shobana, Suresh Gopi]","[Comedy, Horror, Mystery, Thriller]",India,Malayalam
2,Spadikam,1995,150 mins,4.3,Bhadran,A small-town criminal constantly clashes with ...,"[Mohanlal, Thilakan, Urvashi]","[Action, Drama]",India,Malayalam
3,Ustad Hotel,2012,151 mins,4.1,Anwar Rasheed,Faisi wants to go to UK to become a profession...,"[Dulquer Salmaan, Thilakan, Siddique]","[Comedy, Drama]",India,Malayalam
4,Mumbai Police,2013,145 mins,3.95,Rosshan Andrrews,ACP Antony is the only one who knows the truth...,"[Prithviraj Sukumaran, Jayasurya, Rahman]","[Crime, Mystery, Thriller]",India,Malayalam
5,Memories,2013,132 mins,4.0,Jeethu Joseph,"Sam Alex, a drunkard ex-cop, is forced to inve...","[Prithviraj Sukumaran, Sreejith Ravi, Vijayara...","[Crime, Mystery, Thriller]",India,Malayalam
6,Drishyam,2013,160 mins,4.15,Jeethu Joseph,A man goes to extreme lengths to save his fami...,"[Mohanlal, Meena, Asha Sharath]","[Crime, Drama, Thriller]",India,Malayalam
7,Bangalore Days,2014,171 mins,4.15,Anjali Menon,A fun roller coaster ride about three young pe...,"[Dulquer Salmaan, Nivin Pauly, Nazriya Nazim]","[Comedy, Drama, Romance]",India,Malayalam
8,Premam,2015,156 mins,4.15,Alphonse Puthren,A young man has three opportunities to find lo...,"[Nivin Pauly, Sai Pallavi, Madonna Sebastian]","[Comedy, Drama, Romance]",India,Malayalam
9,Charlie,2015,130 mins,4.0,Martin Prakkat,"A young, nonconforming woman named Tessa gets ...","[Dulquer Salmaan, Parvathy Thiruvothu, Aparna ...","[Adventure, Drama, Romance, Thriller]",India,Malayalam
10,Pulimurugan,2016,161 mins,3.35,Vysakh,When a small village near a forest is under th...,"[Mohanlal, Jagapathi Babu, Kamalinee Mukherjee]","[Action, Thriller]",India,Malayalam


In [18]:
# creating a single Data Frame
Indian_movies_dfs = [telugu_movies_df, Bollywood_movies_df, Kannada_movies_df, Tamil_movies_df, Malayalam_movies_df]
Indian_movies_df = pd.concat(Indian_movies_dfs, ignore_index=True)
Indian_movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
0,Okkadu,2003,171 mins,4.0,Gunasekhar,A Kabbadi player rescues a young woman from an...,"[Mahesh Babu, Bhoomika Chawla, Prakash Raj]","[Action, Drama, Romance, Sport]",India,Telugu
1,Arya,2004,151 mins,3.9,Sukumar,"Geethanjali, who is already the girlfriend of ...","[Allu Arjun, Anuradha Mehta, Siva Balaji]","[Action, Comedy, Drama, Romance]",India,Telugu
2,Athadu,2005,172 mins,4.1,Trivikram Srinivas,"A gunman for hire is framed for murder, and as...","[Mahesh Babu, Trisha Krishnan, Sonu Sood]","[Action, Crime, Drama, Mystery, Thriller]",India,Telugu
3,Bommarillu,2006,170 mins,4.1,Bommarillu Baskar,A young man realizes that he has to push back ...,"[Siddharth, Genelia Deshmukh, Prakash Raj]","[Comedy, Drama, Romance]",India,Telugu
4,Pokiri,2006,164 mins,4.0,Puri Jagannadh,A money-minded criminal has to deal with two r...,"[Mahesh Babu, Ileana D'Cruz, Prakash Raj]","[Gun Fu, Action, Crime, Thriller]",India,Telugu
...,...,...,...,...,...,...,...,...,...,...
310,Anweshippin Kandethum,2024,142 mins,3.7,Darwin Kuriakose,Revolves around two major crimes that shook ke...,"[Tovino Thomas, Vineeth Thattil David, Rahul R...","[Crime, Drama, Thriller]",India,Malayalam
311,Manjummel Boys,2024,135 mins,4.1,Chidambaram,A group of friends get into a daring rescue mi...,"[Soubin Shahir, Sreenath Bhasi, Balu Varghese]","[Adventure, Drama, Thriller]",India,Malayalam
312,Bramayugam,2024,139 mins,3.9,Rahul Sadasivan,"Thevan, a folk singer of the Paanan caste, has...","[Mammootty, Arjun Ashokan, Sidharth Bharathan]","[Drama, Horror, Thriller]",India,Malayalam
313,The Goat Life,2024,173 mins,3.55,Blessy,An Indian man seeking work follows a job lead ...,"[Prithviraj Sukumaran, Jimmy Jean-Louis, K.r. ...","[Desert Adventure, Survival, Adventure, Drama]",India,Malayalam


In [19]:
Indian_movies_df.to_csv("Indian_movies_dataset", index=False)

In [20]:
pd.read_csv("Indian_movies_dataset")

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
0,Okkadu,2003,171 mins,4.00,Gunasekhar,A Kabbadi player rescues a young woman from an...,"['Mahesh Babu', 'Bhoomika Chawla', 'Prakash Raj']","['Action', 'Drama', 'Romance', 'Sport']",India,Telugu
1,Arya,2004,151 mins,3.90,Sukumar,"Geethanjali, who is already the girlfriend of ...","['Allu Arjun', 'Anuradha Mehta', 'Siva Balaji']","['Action', 'Comedy', 'Drama', 'Romance']",India,Telugu
2,Athadu,2005,172 mins,4.10,Trivikram Srinivas,"A gunman for hire is framed for murder, and as...","['Mahesh Babu', 'Trisha Krishnan', 'Sonu Sood']","['Action', 'Crime', 'Drama', 'Mystery', 'Thril...",India,Telugu
3,Bommarillu,2006,170 mins,4.10,Bommarillu Baskar,A young man realizes that he has to push back ...,"['Siddharth', 'Genelia Deshmukh', 'Prakash Raj']","['Comedy', 'Drama', 'Romance']",India,Telugu
4,Pokiri,2006,164 mins,4.00,Puri Jagannadh,A money-minded criminal has to deal with two r...,"['Mahesh Babu', ""Ileana D'Cruz"", 'Prakash Raj']","['Gun Fu', 'Action', 'Crime', 'Thriller']",India,Telugu
...,...,...,...,...,...,...,...,...,...,...
310,Anweshippin Kandethum,2024,142 mins,3.70,Darwin Kuriakose,Revolves around two major crimes that shook ke...,"['Tovino Thomas', 'Vineeth Thattil David', 'Ra...","['Crime', 'Drama', 'Thriller']",India,Malayalam
311,Manjummel Boys,2024,135 mins,4.10,Chidambaram,A group of friends get into a daring rescue mi...,"['Soubin Shahir', 'Sreenath Bhasi', 'Balu Varg...","['Adventure', 'Drama', 'Thriller']",India,Malayalam
312,Bramayugam,2024,139 mins,3.90,Rahul Sadasivan,"Thevan, a folk singer of the Paanan caste, has...","['Mammootty', 'Arjun Ashokan', 'Sidharth Bhara...","['Drama', 'Horror', 'Thriller']",India,Malayalam
313,The Goat Life,2024,173 mins,3.55,Blessy,An Indian man seeking work follows a job lead ...,"['Prithviraj Sukumaran', 'Jimmy Jean-Louis', '...","['Desert Adventure', 'Survival', 'Adventure', ...",India,Malayalam
